In [ ]:
import pandas as pd
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#cols=['ID','Diagnosis','radius1','texture1','perimeter1','area1','smoothness1','compactness1','concavity1','concave_points1','symmetry1','fractal_dimension1','radius2','texture2','perimeter2','area2','smoothness2','compactness2','concavity2','concave_points2','symmetry2','fractal_dimension2','radius3','texture3','perimeter3']
df=pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
df.isnull().any()

Age               False
Sex               False
ChestPainType     False
RestingBP         False
Cholesterol       False
FastingBS         False
RestingECG        False
MaxHR             False
ExerciseAngina    False
Oldpeak           False
ST_Slope          False
HeartDisease      False
dtype: bool

In [ ]:
df1=df.drop_duplicates()
df1

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
df2=df1.drop(['Sex'],axis=1)
df2=pd.get_dummies(df[['ChestPainType','RestingECG','ExerciseAngina']],dtype=int)
df2

,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y
0,0,1,0,0,0,1,0,1,0
1,0,0,1,0,0,1,0,1,0
2,0,1,0,0,0,0,1,1,0
3,1,0,0,0,0,1,0,0,1
4,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
913,0,0,0,1,0,1,0,1,0
914,1,0,0,0,0,1,0,1,0
915,1,0,0,0,0,1,0,0,1
916,0,1,0,0,1,0,0,1,0


In [ ]:
df3=pd.concat([df1,df2],axis=1)
df3

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,...,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y
0,40,M,ATA,140,289,0,Normal,172,N,0.0,...,0,0,1,0,0,0,1,0,1,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,...,1,0,0,1,0,0,1,0,1,0
2,37,M,ATA,130,283,0,ST,98,N,0.0,...,0,0,1,0,0,0,0,1,1,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,...,1,1,0,0,0,0,1,0,0,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,...,1,0,0,0,1,0,1,0,1,0
914,68,M,ASY,144,193,1,Normal,141,N,3.4,...,1,1,0,0,0,0,1,0,1,0
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,...,1,1,0,0,0,0,1,0,0,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,...,1,0,1,0,0,1,0,0,1,0


In [ ]:
df4=df3.drop(['Sex','ChestPainType','RestingECG','ExerciseAngina'],axis=1)
df4=df4.drop(['ST_Slope'],axis=1)

df4

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y
0,40,140,289,0,172,0.0,0,0,1,0,0,0,1,0,1,0
1,49,160,180,0,156,1.0,1,0,0,1,0,0,1,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,0,0,0,1,1,0
3,48,138,214,0,108,1.5,1,1,0,0,0,0,1,0,0,1
4,54,150,195,0,122,0.0,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,0,0,0,1,0,1,0,1,0
914,68,144,193,1,141,3.4,1,1,0,0,0,0,1,0,1,0
915,57,130,131,0,115,1.2,1,1,0,0,0,0,1,0,0,1
916,57,130,236,0,174,0.0,1,0,1,0,0,1,0,0,1,0


# **naive base**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
x=df4.drop(['HeartDisease'],axis=1)
#y=df4['HeartDisease']

y=df4['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)
gnb=GaussianNB()
gnb.fit(X_train,y_train)


GaussianNB()

In [ ]:
y_pred=gnb.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.79      0.79        86
           1       0.81      0.81      0.81        98

    accuracy                           0.80       184
   macro avg       0.80      0.80      0.80       184
weighted avg       0.80      0.80      0.80       184



In [ ]:
gnb.score(X_train,y_train)

0.8269754768392371

In [ ]:
gnb.score(X_test,y_test)

0.7989130434782609

# **knn**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train,y_train)

KNeighborsClassifier()

In [ ]:
y_pred=knn_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.65      0.69        86
           1       0.72      0.80      0.76        98

    accuracy                           0.73       184
   macro avg       0.73      0.72      0.72       184
weighted avg       0.73      0.73      0.73       184



In [ ]:
knn_model.score(X_train,y_train)

0.7874659400544959

In [ ]:
knn_model.score(X_test,y_test)

0.7282608695652174

# **svm**

In [ ]:
from sklearn.svm import SVC
svm_model=SVC()
svm_model=svm_model.fit(X_train,y_train)

In [ ]:
y_pred=svm_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.64      0.67        86
           1       0.71      0.77      0.74        98

    accuracy                           0.71       184
   macro avg       0.71      0.70      0.70       184
weighted avg       0.71      0.71      0.71       184



In [ ]:
svm_model.score(X_train,y_train)

0.7343324250681199

In [ ]:
svm_model.score(X_test,y_test)

0.7065217391304348

# **log reg**

In [ ]:
from sklearn.linear_model import LogisticRegression
lg_model=LogisticRegression()
lg_model=lg_model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
y_pred=lg_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.76      0.78        86
           1       0.80      0.84      0.82        98

    accuracy                           0.80       184
   macro avg       0.80      0.80      0.80       184
weighted avg       0.80      0.80      0.80       184



In [ ]:
lg_model.score(X_train,y_train)

0.8119891008174387

In [ ]:
lg_model.score(X_test,y_test)

0.7989130434782609